In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [9]:
df=pd.read_csv(r'C:\Users\Ganesh\Desktop\jupiter\neuron\Project\Sentiment Analysis\train.csv')

In [10]:
df.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [12]:
# cheaking the shape
df.shape

(7920, 3)

In [13]:
#Cheacking the values of label
df.label.value_counts()


0    5894
1    2026
Name: label, dtype: int64

In [14]:
# cheacking the null values
df.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [15]:
# droping id column as not needed 
df=df.drop('id', axis=1)

In [16]:
df.shape

(7920, 2)

In [17]:
# spliting the data into independent and dependent features
x=df['tweet']
y=df['label']

In [91]:
x.shape

(7920,)

In [26]:
# printing some revies

x0=x[0:10]
print(x0)
print('='*100)
x500=x[500:510]
print(x500)
print('='*100)
x2000=x[2000:2010]
print(x2000)
print('='*100)

0    #fingerprint #Pregnancy Test https://goo.gl/h1...
1    Finally a transparant silicon case ^^ Thanks t...
2    We love this! Would you go? #talk #makememorie...
3    I'm wired I know I'm George I was made that wa...
4    What amazing service! Apple won't even talk to...
5    iPhone software update fucked up my phone big ...
6    Happy for us .. #instapic #instadaily #us #son...
7    New Type C charger cable #UK http://www.ebay.c...
8    Bout to go shopping again listening to music #...
9    Photo: #fun #selfie #pool #water #sony #camera...
Name: tweet, dtype: object
500    Set up almost done :)) #NewTv #Samsung #Black ...
501    The main reason why I #hateApple is the IOS bu...
502    How do you catch up with your family....? #sou...
503    Even if it was just a picture of an apple. #ap...
504    New cases hanogramcase martyarchs #cases #name...
505    @architecture_3design - TAG YOUR FRIENDS @arch...
506    Iphone 5 is just like the 4s but thinner and h...
507    $&@*# you Apple. 

# Using Lemmatizer

In [40]:
import re
from tqdm import tqdm
import nltk
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
corpus=[]
for i in tqdm(range(len(x))):
    review=re.sub(r"http\S+", "", x[i])
    review=re.sub(r"[^a-zA-Z]",' ',review)
    review=review.lower()
    review=review.split()
    review=[lemmatizer.lemmatize(x) for x in review if not x in stopwords.words('english')]
    review=' '.join(review)
    corpus.append(review)
    


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ganesh\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.
100%|██████████████████████████████████████████████████████████████████████████████| 7920/7920 [01:25<00:00, 92.50it/s]


In [90]:
len(corpus)

15840

In [43]:
# importing tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
X=tf.fit_transform(corpus).toarray()

In [44]:
X.shape

(7920, 16674)

In [45]:
y.shape

(7920,)

In [47]:

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X,y,test_size=0.25,random_state=42)

In [50]:
# cheacking some features name
tf.get_feature_names()[:10]

['aa',
 'aaaahhhhhhh',
 'aag',
 'aah',
 'aalborg',
 'aand',
 'aapl',
 'aarhus',
 'aaron',
 'aaronbrandt']

In [51]:
#cheacking the parametres
tf.get_params()

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

In [52]:
# now we importing naive_bayes model
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
nvb=MultinomialNB()
nvb.fit(xtrain,ytrain)
pred=nvb.predict(xtest)
acc=accuracy_score(ytest,pred)*float(100)
acc

82.77777777777777

In [53]:
# ploting the confussion matrix
from sklearn.metrics import confusion_matrix,plot_confusion_matrix

con=confusion_matrix(ytest,pred)
con

array([[1414,   25],
       [ 316,  225]], dtype=int64)

In [54]:
#some hyperparameter tunnig
previous_score=0
for alpha in np.arange(0,1,0.1):
    nvb=MultinomialNB(alpha=alpha)
    nvb.fit(xtrain,ytrain)
    pred=nvb.predict(xtest)
    score = accuracy_score(ytest, pred)
    if score>previous_score:
        classifier=nvb
    print("Alpha: {}, Score : {}".format(alpha,score))

C:\python\lib\site-packages\sklearn\naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha: 0.0, Score : 0.8515151515151516
Alpha: 0.1, Score : 0.8747474747474747
Alpha: 0.2, Score : 0.8787878787878788
Alpha: 0.30000000000000004, Score : 0.8767676767676768
Alpha: 0.4, Score : 0.8707070707070707
Alpha: 0.5, Score : 0.8651515151515151
Alpha: 0.6000000000000001, Score : 0.8626262626262626
Alpha: 0.7000000000000001, Score : 0.851010101010101
Alpha: 0.8, Score : 0.8409090909090909
Alpha: 0.9, Score : 0.8353535353535354


In [55]:
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
nvb=MultinomialNB(alpha=0.3)
nvb.fit(xtrain,ytrain)
pred=nvb.predict(xtest)
acc=accuracy_score(ytest,pred)*float(100)
acc

87.67676767676768

# Using Stemming

In [64]:
from nltk.stem.porter import PorterStemmer
porter=PorterStemmer()
corpus1=[]
for i in tqdm(range(len(x))):
    review=re.sub(r"http\S+", "", x[i])
    review=re.sub(r"[^a-zA-Z]",' ',review)
    review=review.lower()
    review=review.split()
    review=[porter.stem(x) for x in review if not x in stopwords.words('english')]
    review=' '.join(review)
    corpus1.append(review)

100%|██████████████████████████████████████████████████████████████████████████████| 7920/7920 [01:27<00:00, 90.05it/s]


In [107]:
len(corpus1)

7920

In [66]:
# importing tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
tf=TfidfVectorizer()
X1=tf.fit_transform(corpus1).toarray()

In [67]:

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(X1,y,test_size=0.25,random_state=42)

In [68]:
# now we importing naive_bayes model
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
nvb=MultinomialNB()
nvb.fit(xtrain,ytrain)
pred=nvb.predict(xtest)
acc=accuracy_score(ytest,pred)*float(100)
acc

82.67676767676768

In [69]:
#some hyperparameter tunnig
previous_score=0
for alpha in np.arange(0,1,0.1):
    nvb=MultinomialNB(alpha=alpha)
    nvb.fit(xtrain,ytrain)
    pred=nvb.predict(xtest)
    score = accuracy_score(ytest, pred)
    if score>previous_score:
        classifier=nvb
    print("Alpha: {}, Score : {}".format(alpha,score))

C:\python\lib\site-packages\sklearn\naive_bayes.py:512: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


Alpha: 0.0, Score : 0.85
Alpha: 0.1, Score : 0.8691919191919192
Alpha: 0.2, Score : 0.8787878787878788
Alpha: 0.30000000000000004, Score : 0.8757575757575757
Alpha: 0.4, Score : 0.8712121212121212
Alpha: 0.5, Score : 0.8641414141414141
Alpha: 0.6000000000000001, Score : 0.8570707070707071
Alpha: 0.7000000000000001, Score : 0.8459595959595959
Alpha: 0.8, Score : 0.8444444444444444
Alpha: 0.9, Score : 0.8338383838383838


In [70]:
# now we importing naive_bayes model
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
nvb=MultinomialNB(alpha=0.3)
nvb.fit(xtrain,ytrain)
pred=nvb.predict(xtest)
acc=accuracy_score(ytest,pred)*float(100)
acc

87.57575757575758

#not much differance seen in accuracy using stemming and lemmatization

# Using  deep learning technique LSTM

In [72]:
from tensorflow.keras.layers import Embedding,Dense,LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences 

In [81]:
# first defind dictionory size

vec_size=5000


In [108]:
# maximum sentence len in corpus
le=[]
for i in corpus1:
    le.append(len(i.split()))
s=max(le)
s

39

In [79]:
# defined sentence length
sen_length=39

In [109]:
len(corpus1)

7920

Onehot representation

In [110]:
onehot=[one_hot(words,vec_size) for words in corpus1]

In [111]:
# getting index form dictionory
print(onehot)

[[831, 1016, 2017, 1033, 768, 2173, 664, 2153, 4377, 3009, 2074, 1265], [3901, 3719, 3975, 2027, 1021, 1271, 3148, 3228, 3371, 3316], [4573, 3261, 3269, 4667, 4014, 3492, 1852, 1265, 2429, 3040, 2412], [3295, 705, 4835, 4165, 1208, 1265, 664, 4320, 1200], [2286, 3442, 1688, 730, 4667, 1730, 1502, 910, 2520, 428], [1265, 3610, 1434, 3315, 1651, 2102, 814, 2520, 1265], [4589, 1674, 22, 3632, 1674, 3228, 3371, 1350], [3903, 4586, 2641, 1684, 2055, 3402, 3077, 1360, 4305, 3903, 4487, 1707, 4762, 2249, 4649, 1544, 832, 3985, 2469, 274, 2098, 2317, 541, 4783, 4065, 4361, 3048, 158, 2344, 3452], [2074, 3269, 3822, 3626, 4240, 1265, 2785, 4240, 1253, 2144], [4514, 3508, 3725, 4887, 3185, 3228, 4061, 3465, 2891, 1765, 4359, 664, 1696], [3722, 1688, 3258, 3903, 3035, 2086, 3258, 3903, 3988, 1755, 228, 3258, 2036, 925, 2271, 4536, 4312], [1847, 2606, 4274, 4354, 1688, 4263, 3377, 2741, 1696, 3436], [3896, 2128, 1265, 3400, 3677, 3599, 3903, 1033, 1033, 3691], [4165, 3662, 2246, 2947, 374, 2664, 3

In [112]:
len(onehot)

7920

Word Embedding representation

In [113]:
# to get equal length we use padding
seq_pad=pad_sequences(onehot,padding='pre',maxlen=39)
print(seq_pad)

[[   0    0    0 ... 3009 2074 1265]
 [   0    0    0 ... 3228 3371 3316]
 [   0    0    0 ... 2429 3040 2412]
 ...
 [   0    0    0 ...  388 4354 4303]
 [   0    0    0 ...  105 1347 4952]
 [   0    0    0 ...  549 3508 2992]]


In [114]:
# define the feature size i.e diamention
dim=40

In [115]:
# initialize the model
model=Sequential()
model.add(Embedding(vec_size,dim,input_length=sen_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 39, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [116]:
len(onehot),y.shape

(7920, (7920,))

In [118]:
# convert onehot into array
import numpy as np
X_final=np.array(seq_pad)
y_final=np.array(y)

In [121]:
X_final.shape,y_final.shape

((7920, 39), (7920,))

In [122]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=20)

In [123]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Train on 5544 samples, validate on 2376 samples
Epoch 1/10
5544/5544 [==============================] - ETA: 43:14 - loss: 0.6926 - accuracy: 0.531 - ETA: 21:25 - loss: 0.6903 - accuracy: 0.640 - ETA: 14:08 - loss: 0.6880 - accuracy: 0.666 - ETA: 10:30 - loss: 0.6844 - accuracy: 0.691 - ETA: 8:19 - loss: 0.6821 - accuracy: 0.693 - ETA: 6:52 - loss: 0.6777 - accuracy: 0.71 - ETA: 5:49 - loss: 0.6727 - accuracy: 0.72 - ETA: 5:03 - loss: 0.6698 - accuracy: 0.71 - ETA: 4:26 - loss: 0.6642 - accuracy: 0.72 - ETA: 3:57 - loss: 0.6593 - accuracy: 0.72 - ETA: 3:33 - loss: 0.6466 - accuracy: 0.73 - ETA: 3:14 - loss: 0.6420 - accuracy: 0.73 - ETA: 2:57 - loss: 0.6284 - accuracy: 0.74 - ETA: 2:42 - loss: 0.6419 - accuracy: 0.73 - ETA: 2:30 - loss: 0.6386 - accuracy: 0.73 - ETA: 2:19 - loss: 0.6276 - accuracy: 0.74 - ETA: 2:09 - loss: 0.6235 - accuracy: 0.74 - ETA: 2:00 - loss: 0.6260 - accuracy: 0.73 - ETA: 1:53 - loss: 0.6224 - accuracy: 0.73 - ETA: 1:46 - loss: 0.6202 - accuracy: 0.73 - ETA: 1:

5544/5544 [==============================] - ETA: 8s - loss: 0.1556 - accuracy: 0.93 - ETA: 8s - loss: 0.1520 - accuracy: 0.93 - ETA: 9s - loss: 0.1521 - accuracy: 0.94 - ETA: 8s - loss: 0.1439 - accuracy: 0.94 - ETA: 9s - loss: 0.1530 - accuracy: 0.94 - ETA: 8s - loss: 0.1661 - accuracy: 0.94 - ETA: 8s - loss: 0.1602 - accuracy: 0.94 - ETA: 8s - loss: 0.1680 - accuracy: 0.93 - ETA: 8s - loss: 0.1746 - accuracy: 0.93 - ETA: 7s - loss: 0.1747 - accuracy: 0.93 - ETA: 7s - loss: 0.1713 - accuracy: 0.93 - ETA: 7s - loss: 0.1730 - accuracy: 0.93 - ETA: 7s - loss: 0.1759 - accuracy: 0.93 - ETA: 7s - loss: 0.1840 - accuracy: 0.92 - ETA: 6s - loss: 0.1786 - accuracy: 0.93 - ETA: 6s - loss: 0.1773 - accuracy: 0.93 - ETA: 6s - loss: 0.1739 - accuracy: 0.93 - ETA: 6s - loss: 0.1727 - accuracy: 0.93 - ETA: 6s - loss: 0.1710 - accuracy: 0.93 - ETA: 6s - loss: 0.1726 - accuracy: 0.93 - ETA: 6s - loss: 0.1692 - accuracy: 0.93 - ETA: 6s - loss: 0.1732 - accuracy: 0.93 - ETA: 5s - loss: 0.1705 - accura

5544/5544 [==============================] - ETA: 9s - loss: 0.1353 - accuracy: 0.96 - ETA: 8s - loss: 0.1204 - accuracy: 0.96 - ETA: 8s - loss: 0.0988 - accuracy: 0.97 - ETA: 8s - loss: 0.0875 - accuracy: 0.97 - ETA: 8s - loss: 0.0871 - accuracy: 0.97 - ETA: 7s - loss: 0.0829 - accuracy: 0.97 - ETA: 7s - loss: 0.0808 - accuracy: 0.97 - ETA: 7s - loss: 0.0898 - accuracy: 0.97 - ETA: 7s - loss: 0.0864 - accuracy: 0.97 - ETA: 7s - loss: 0.0873 - accuracy: 0.97 - ETA: 7s - loss: 0.0905 - accuracy: 0.97 - ETA: 6s - loss: 0.0887 - accuracy: 0.97 - ETA: 6s - loss: 0.0859 - accuracy: 0.97 - ETA: 6s - loss: 0.0841 - accuracy: 0.97 - ETA: 6s - loss: 0.0817 - accuracy: 0.97 - ETA: 6s - loss: 0.0814 - accuracy: 0.97 - ETA: 6s - loss: 0.0823 - accuracy: 0.97 - ETA: 6s - loss: 0.0813 - accuracy: 0.97 - ETA: 6s - loss: 0.0827 - accuracy: 0.97 - ETA: 5s - loss: 0.0847 - accuracy: 0.97 - ETA: 5s - loss: 0.0832 - accuracy: 0.97 - ETA: 5s - loss: 0.0808 - accuracy: 0.97 - ETA: 5s - loss: 0.0879 - accura

5544/5544 [==============================] - ETA: 6s - loss: 0.0977 - accuracy: 0.96 - ETA: 6s - loss: 0.0610 - accuracy: 0.98 - ETA: 6s - loss: 0.0520 - accuracy: 0.98 - ETA: 7s - loss: 0.0569 - accuracy: 0.98 - ETA: 7s - loss: 0.0660 - accuracy: 0.98 - ETA: 7s - loss: 0.0679 - accuracy: 0.98 - ETA: 6s - loss: 0.0652 - accuracy: 0.98 - ETA: 6s - loss: 0.0600 - accuracy: 0.98 - ETA: 6s - loss: 0.0580 - accuracy: 0.98 - ETA: 6s - loss: 0.0633 - accuracy: 0.98 - ETA: 6s - loss: 0.0678 - accuracy: 0.98 - ETA: 6s - loss: 0.0650 - accuracy: 0.98 - ETA: 6s - loss: 0.0672 - accuracy: 0.98 - ETA: 6s - loss: 0.0764 - accuracy: 0.97 - ETA: 6s - loss: 0.0745 - accuracy: 0.97 - ETA: 6s - loss: 0.0711 - accuracy: 0.97 - ETA: 6s - loss: 0.0719 - accuracy: 0.97 - ETA: 5s - loss: 0.0698 - accuracy: 0.98 - ETA: 5s - loss: 0.0683 - accuracy: 0.98 - ETA: 5s - loss: 0.0707 - accuracy: 0.98 - ETA: 5s - loss: 0.0686 - accuracy: 0.98 - ETA: 5s - loss: 0.0682 - accuracy: 0.98 - ETA: 5s - loss: 0.0672 - accura

5544/5544 [==============================] - ETA: 7s - loss: 0.0172 - accuracy: 1.00 - ETA: 8s - loss: 0.0246 - accuracy: 0.99 - ETA: 8s - loss: 0.0359 - accuracy: 0.98 - ETA: 7s - loss: 0.0546 - accuracy: 0.98 - ETA: 7s - loss: 0.0594 - accuracy: 0.97 - ETA: 7s - loss: 0.0659 - accuracy: 0.97 - ETA: 7s - loss: 0.0604 - accuracy: 0.97 - ETA: 6s - loss: 0.0538 - accuracy: 0.98 - ETA: 6s - loss: 0.0521 - accuracy: 0.98 - ETA: 6s - loss: 0.0562 - accuracy: 0.97 - ETA: 6s - loss: 0.0593 - accuracy: 0.97 - ETA: 6s - loss: 0.0564 - accuracy: 0.98 - ETA: 6s - loss: 0.0556 - accuracy: 0.98 - ETA: 6s - loss: 0.0535 - accuracy: 0.98 - ETA: 6s - loss: 0.0530 - accuracy: 0.98 - ETA: 6s - loss: 0.0519 - accuracy: 0.98 - ETA: 6s - loss: 0.0496 - accuracy: 0.98 - ETA: 6s - loss: 0.0478 - accuracy: 0.98 - ETA: 5s - loss: 0.0476 - accuracy: 0.98 - ETA: 5s - loss: 0.0476 - accuracy: 0.98 - ETA: 5s - loss: 0.0479 - accuracy: 0.98 - ETA: 5s - loss: 0.0478 - accuracy: 0.98 - ETA: 5s - loss: 0.0466 - accura

In [124]:
y_pred=model.predict_classes(X_test)
confusion_matrix(y_test,y_pred)

array([[1579,  193],
       [ 184,  420]], dtype=int64)

In [125]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8413299663299664

In [137]:
#Adding Dropout and nueron
from tensorflow.keras.layers import Dropout
## Creating model
model=Sequential()
model.add(Embedding(vec_size,dim,input_length=sen_length))
model.add(Dropout(0.3))
model.add(LSTM(200))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 39, 40)            200000    
_________________________________________________________________
dropout_4 (Dropout)          (None, 39, 40)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 200)               192800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 201       
Total params: 393,001
Trainable params: 393,001
Non-trainable params: 0
_________________________________________________________________
None


In [138]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64)

Train on 5544 samples, validate on 2376 samples
Epoch 1/5
5544/5544 [==============================] - ETA: 6:03 - loss: 0.6932 - accuracy: 0.53 - ETA: 3:06 - loss: 0.6891 - accuracy: 0.64 - ETA: 2:06 - loss: 0.6851 - accuracy: 0.68 - ETA: 1:37 - loss: 0.6781 - accuracy: 0.71 - ETA: 1:20 - loss: 0.6710 - accuracy: 0.73 - ETA: 1:08 - loss: 0.6634 - accuracy: 0.73 - ETA: 59s - loss: 0.6467 - accuracy: 0.7388 - ETA: 53s - loss: 0.6680 - accuracy: 0.736 - ETA: 48s - loss: 0.6629 - accuracy: 0.736 - ETA: 43s - loss: 0.6585 - accuracy: 0.732 - ETA: 40s - loss: 0.6538 - accuracy: 0.731 - ETA: 37s - loss: 0.6485 - accuracy: 0.734 - ETA: 35s - loss: 0.6447 - accuracy: 0.738 - ETA: 33s - loss: 0.6407 - accuracy: 0.744 - ETA: 31s - loss: 0.6396 - accuracy: 0.743 - ETA: 29s - loss: 0.6392 - accuracy: 0.742 - ETA: 27s - loss: 0.6383 - accuracy: 0.741 - ETA: 26s - loss: 0.6380 - accuracy: 0.739 - ETA: 25s - loss: 0.6368 - accuracy: 0.739 - ETA: 24s - loss: 0.6337 - accuracy: 0.743 - ETA: 23s - loss:

5544/5544 [==============================] - ETA: 12s - loss: 0.1847 - accuracy: 0.953 - ETA: 12s - loss: 0.1836 - accuracy: 0.953 - ETA: 13s - loss: 0.2001 - accuracy: 0.932 - ETA: 13s - loss: 0.2040 - accuracy: 0.918 - ETA: 13s - loss: 0.2010 - accuracy: 0.912 - ETA: 12s - loss: 0.1841 - accuracy: 0.924 - ETA: 12s - loss: 0.1873 - accuracy: 0.921 - ETA: 12s - loss: 0.1816 - accuracy: 0.923 - ETA: 12s - loss: 0.1785 - accuracy: 0.925 - ETA: 12s - loss: 0.1724 - accuracy: 0.929 - ETA: 12s - loss: 0.1642 - accuracy: 0.934 - ETA: 11s - loss: 0.1679 - accuracy: 0.933 - ETA: 11s - loss: 0.1720 - accuracy: 0.931 - ETA: 11s - loss: 0.1687 - accuracy: 0.934 - ETA: 11s - loss: 0.1712 - accuracy: 0.932 - ETA: 11s - loss: 0.1804 - accuracy: 0.928 - ETA: 11s - loss: 0.1757 - accuracy: 0.931 - ETA: 10s - loss: 0.1770 - accuracy: 0.930 - ETA: 10s - loss: 0.1788 - accuracy: 0.929 - ETA: 10s - loss: 0.1853 - accuracy: 0.925 - ETA: 10s - loss: 0.1917 - accuracy: 0.922 - ETA: 10s - loss: 0.1981 - accur

5544/5544 [==============================] - ETA: 17s - loss: 0.1572 - accuracy: 0.953 - ETA: 15s - loss: 0.1729 - accuracy: 0.953 - ETA: 14s - loss: 0.1486 - accuracy: 0.958 - ETA: 14s - loss: 0.1256 - accuracy: 0.960 - ETA: 13s - loss: 0.1273 - accuracy: 0.956 - ETA: 13s - loss: 0.1197 - accuracy: 0.958 - ETA: 13s - loss: 0.1125 - accuracy: 0.962 - ETA: 13s - loss: 0.1062 - accuracy: 0.966 - ETA: 13s - loss: 0.1031 - accuracy: 0.967 - ETA: 12s - loss: 0.1057 - accuracy: 0.965 - ETA: 12s - loss: 0.0977 - accuracy: 0.968 - ETA: 12s - loss: 0.0954 - accuracy: 0.968 - ETA: 12s - loss: 0.0947 - accuracy: 0.968 - ETA: 12s - loss: 0.1001 - accuracy: 0.966 - ETA: 12s - loss: 0.1014 - accuracy: 0.965 - ETA: 11s - loss: 0.1040 - accuracy: 0.963 - ETA: 11s - loss: 0.1064 - accuracy: 0.964 - ETA: 11s - loss: 0.1085 - accuracy: 0.962 - ETA: 11s - loss: 0.1089 - accuracy: 0.962 - ETA: 11s - loss: 0.1109 - accuracy: 0.960 - ETA: 10s - loss: 0.1096 - accuracy: 0.961 - ETA: 10s - loss: 0.1132 - accur

In [139]:
y_pred=model.predict_classes(X_test)

In [140]:
y_pred=model.predict_classes(X_test)
confusion_matrix(y_test,y_pred)

array([[1613,  159],
       [ 158,  446]], dtype=int64)

In [141]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8665824915824916